In [1]:
import json
import requests
import pandas as pd

def watson_request(text):
    base_url = "https://gateway.watsonplatform.net/tone-analyzer-experimental/api/v1/tone"
    r = requests.post(base_url, data = text, headers = {"Content-Type" : "text/plain"}, 
                      auth=("84a3bf89-c5f3-45b6-9ee2-5c6ee1d66fa5", "t95US4RGaQnX"))
    response = json.loads(r.text)
    return response

In [2]:
def read_transcript(gfile):
    hillary = []
    sanders = []
    result = []
    i = 0
    while True:
        line = gfile.readline()
        if line.startswith("**"):
            break
        if line.startswith("CLINTON:"):
            text = line.split("CLINTON:")[1]
            inline = gfile.readline()
            while True: 
                inline = gfile.readline()
                if (inline.startswith("(") or inline.startswith("TODD:") or inline.startswith("SANDERS:") or inline.startswith("MADDOW:")) :
                    break
                text = text+" " +inline
           
            result.append({"id" : "hillary", "text": text} )
            
        if line.startswith("SANDERS:"):
            text = line.split(":")[1]
            
            while True:
                inline = gfile.readline()
                if (inline.startswith("(") or inline.startswith("TODD:") or inline.startswith("CLINTON:") or inline.startswith("MADDOW:") ):
                    break
                text = text+" " +inline
            result.append({"id" : "sanders", "text": text} )
            
    return result

In [3]:
line = "CLINTON: hola"
line.split("CLINTON:")[1]

' hola'

In [4]:
f = open('./data/transcript.txt')
transcript = read_transcript(f)

In [5]:
print transcript

[{'text': " Rachel, thank you very much.\n Millions of Americans are giving up on the political process. And they're giving up on the political process because they understand the economy is rigged.\n They are working longer hours for low wages. They're worried about the future of their kids, and yet almost all new income and wealth is going to the top 1 percent. Not what America is supposed to be about. Not the fairness that we grew up believing that America was about. And then sustaining that rigged economy is a corrupt campaign finance system undermining American democracy, where billionaire, Wall Street, corporate America can contribute unlimited sums of money into super PACs and into candidates.\n Our job, together, is to end a rigged economy, create an economy that works for all, and absolutely overturn Citizens United. One person, one vote. That's what American democracy is about.\n", 'id': 'sanders'}, {'text': " Well, I'm happy to be here in New Hampshire for this debate, as we

In [6]:
get_

NameError: name 'get_' is not defined

In [117]:
print len(h)
print len(s)

43
54


In [7]:
def create_row(name_id, text):
    row = {}
    
    watson_response = watson_request(text)
    row["id"] = name_id
    row["text"] = text
    for tone in watson_response['children']:
        tone_id = tone['id']
        for features in tone['children']:
            characteristic_id = tone_id + "_" + features['id']
            row[characteristic_id + "_" + "normalized_score"] = features['normalized_score']
    return row

def analyze_text(transcript):
    response = []
    for line in transcript:
            name_id = line['id']
            text = line['text']
            response.append(create_row(name_id, text))
    return response

In [8]:
analyzed_text = analyze_text(transcript)

In [120]:
analyzed_text


[{u'emotion_tone_Anger_normalized_score': 0.0,
  u'emotion_tone_Cheerfulness_normalized_score': 0.7869793571988503,
  u'emotion_tone_Negative_normalized_score': 0.9590176901288012,
  'id': 'sanders',
  u'social_tone_Agreeableness_Big5_normalized_score': 0.751077735293086,
  u'social_tone_Conscientiousness_Big5_normalized_score': 0.5037829843952596,
  u'social_tone_Openness_Big5_normalized_score': 0.19722877464661942,
  'text': " Rachel, thank you very much.\n Millions of Americans are giving up on the political process. And they're giving up on the political process because they understand the economy is rigged.\n They are working longer hours for low wages. They're worried about the future of their kids, and yet almost all new income and wealth is going to the top 1 percent. Not what America is supposed to be about. Not the fairness that we grew up believing that America was about. And then sustaining that rigged economy is a corrupt campaign finance system undermining American democr

In [1]:
from __future__ import print_function
from alchemyapi import AlchemyAPI
import json


# Create the AlchemyAPI Object
alchemyapi = AlchemyAPI()
def watson_entity(demo_text):
    #print('Processing text: ', demo_text)
    #print('')

    response = alchemyapi.entities('text', demo_text, {'sentiment': 1})

    if response['status'] == 'OK':
        #print('## Response Object ##')
        #print(json.dumps(response, indent=4))

        #print('')
        #print('## Entities ##')
        row = {}
        for entity in response['entities']:
            #print('text: ', entity['text'].encode('utf-8'))
            #print('type: ', entity['type'])
            #print('relevance: ', entity['relevance'])
            #print('sentiment: ', entity['sentiment']['type'])
            if 'score' in entity['sentiment']:
                a = 0
                #print('sentiment score: ' + entity['sentiment']['score'])
            #print('')
    else:
        print('Error in entity extraction call: ', response['statusInfo'])
        print(response)
    return(response)


In [4]:
def create_row(name_id, text):
    row = {}
    
    watson_response = watson_entity(text)
    row["id"] = name_id
    row["text_speach"] = text
    row["entities"] = {}   
    for entity in watson_response['entities']:  
            name = entity['text'].encode('utf-8')
            row["entities"][name]={}
            row["entities"][name]["type"] = entity['type']
            row["entities"][name]["rel"] =  entity['relevance']
            row["entities"][name]["sent"] = entity['sentiment']['type']
            if 'score' in entity['sentiment']:
                row["entities"][name]["sent.score"] = entity['sentiment']['score']
            else:
                row["entities"][name]["sent.score"] = 0
    return row


def analyze_text(transcript):
    response = []
    for line in transcript:
            name_id = line['id']
            text = line['text']
            response.append(create_row(name_id, text))
    return response

In [62]:
analyzed_text=analyze_text(transcript)

Error in entity extraction call:  daily-transaction-limit-exceeded
{u'status': u'ERROR', u'statusInfo': u'daily-transaction-limit-exceeded'}


KeyError: 'entities'

In [44]:
analyzed_text


[{'entities': {'Apple Store': {'rel': u'0.884345',
    'sent': u'negative',
    'sent.score': u'-0.582727',
    'type': u'FieldTerminology'},
   'Bob': {'rel': u'0.617745',
    'sent': u'negative',
    'sent.score': u'-0.508233',
    'type': u'Person'},
   'Colorado': {'rel': u'0.533694',
    'sent': u'neutral',
    'sent.score': 0,
    'type': u'StateOrCounty'},
   'Denver': {'rel': u'0.624176',
    'sent': u'negative',
    'sent.score': u'-0.508233',
    'type': u'City'},
   'iPhone': {'rel': u'0.728395',
    'sent': u'negative',
    'sent.score': u'-0.508233',
    'type': u'Technology'}},
  'id': 'sanders',
  'text_speach': " Rachel, thank you very much.\n Millions of Americans are giving up on the political process. And they're giving up on the political process because they understand the economy is rigged.\n They are working longer hours for low wages. They're worried about the future of their kids, and yet almost all new income and wealth is going to the top 1 percent. Not what 